In [1]:
from sklearn.metrics import precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from joblib import load
from sklearn.utils import resample
import os
import random
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
from PIL import Image
from skimage import color, filters
import cv2
import matplotlib.pyplot as plt
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.draw import circle_perimeter
import urllib.request
from skimage import measure
import joblib
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import f1_score, confusion_matrix, classification_report


In [2]:
def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2
    intersection_area = max(0, min(x2, x4) - max(x1, x3)) * max(0, min(y2, y4) - max(y1, y3))
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x4 - x3) * (y4 - y3)
    union_area = box1_area + box2_area - intersection_area
    iou = intersection_area / union_area
    return iou

In [3]:


def calculate_precision_recall(gt_boxes, pred_boxes, confidence_scores):
    # Calculate precision and recall for a single class
    num_gt_boxes = len(gt_boxes)
    num_pred_boxes = len(pred_boxes)

    true_positives = np.zeros(num_pred_boxes)
    false_positives = np.zeros(num_pred_boxes)
    matched = [False for i in range(num_gt_boxes)]

    for i in range(num_pred_boxes):
        pred_box = pred_boxes[i]
        max_iou = 0
        max_iou_index = -1

        gt_box = gt_boxes[i]
        iou = calculate_iou(pred_box, gt_box)

        if iou  >= 0.5:
            if confidence_scores[i] > 0.5:  # Set confidence threshold here
                if not matched[max_iou_index]:  # If the GT box is not matched yet
                    true_positives[i] = 1
                    matched[max_iou_index] = True  # Mark GT box as matched
                else:
                    false_positives[i] = 1
        else:
            false_positives[i] = 1

    cum_true_positives = np.cumsum(true_positives)
    cum_false_positives = np.cumsum(false_positives)
    precision = cum_true_positives / (cum_true_positives + cum_false_positives)
    recall = cum_true_positives / num_gt_boxes

    return precision, recall

In [4]:
def calculate_ap(precision, recall):
    # Calculate Average Precision (AP) using precision and recall values
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))

    for i in range(len(mpre) - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    i = np.where(mrec[1:] != mrec[:-1])[0]
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])

    return ap


In [5]:

def evaluate_traffic_light_detection(gt_annotations_list, pred_annotations_list):
    # Evaluate traffic light detection using ground truth and predicted annotations

    classes = ['red', 'yellow', 'green']
    ap_values = []

    for cls in classes:
        gt_boxes_all_images = []
        pred_boxes_all_images = []
        confidence_scores_all_images = []

        for i in range(len(gt_annotations_list)):
            gt_annotations = gt_annotations_list[i]
            pred_annotations = pred_annotations_list[i]

            gt_boxes = np.array([box[1:5] for box in gt_annotations if box[5] == cls])
            pred_boxes = np.array([box[1:5] for box in pred_annotations if box[5] == cls])
            confidence_scores = np.array([box[6] for box in pred_annotations if box[5] == cls])

            gt_matched = np.zeros(len(gt_boxes))

            if len(gt_boxes) > 0:
                for j in range(len(pred_boxes)):
                    pred_box = pred_boxes[j]
                    max_iou = 0
                    max_iou_index = -1

                    for k in range(len(gt_boxes)):
                        gt_box = gt_boxes[k]
                        iou = calculate_iou(pred_box, gt_box)

                        if iou > max_iou:
                            max_iou = iou
                            max_iou_index = k

                    if max_iou >= 0.5 and not gt_matched[max_iou_index]:
                        gt_matched[max_iou_index] = 1

                        gt_boxes_all_images.append(gt_boxes[max_iou_index])
                        pred_boxes_all_images.append(pred_box)
                        confidence_scores_all_images.append(confidence_scores[j])

        gt_boxes_all_images = np.array(gt_boxes_all_images)
        pred_boxes_all_images = np.array(pred_boxes_all_images)
        confidence_scores_all_images = np.array(confidence_scores_all_images)
        
        precision, recall = calculate_precision_recall(gt_boxes_all_images, pred_boxes_all_images, confidence_scores_all_images)

        ap = calculate_ap(precision, recall)

        ap_values.append(ap)

    mAP = np.mean(ap_values)
    print("mAP@0.5: ", mAP)

In [6]:
target_classes = ['go', 'stop', 'warning']
color_map = {'go':'green', 'stop':'red', 'warning':'yellow'}

train_folder_list = [
    # 'dayTrain',
    'daySequence1',
    'daySequence2',
#     'sample-dayClip6',
    # 'nightTrain',
    # 'nightSequence1',
    # 'nightSequence2',
#     'sample-nightClip1',
]
# test_percentage = 0.1

In [7]:
def get_annotarion_dataframe(train_data_folders):
    data_base_path = '../input/lisa-traffic-light-dataset/'
    annotation_list = list()
    for folder in [folder + '/' for folder in train_data_folders if os.listdir(data_base_path)]:
        annotation_path = ''
        if 'sample' not in folder:
            annotation_path = data_base_path + 'Annotations/Annotations/' + folder
        else:
            annotation_path = data_base_path + folder*2
        image_frame_path = data_base_path + folder*2
        
        df = pd.DataFrame()
        if 'Clip' in os.listdir(annotation_path)[0]:
            clip_list = os.listdir(annotation_path)
            for clip_folder in clip_list:
                df = pd.read_csv(annotation_path + clip_folder +  '/frameAnnotationsBOX.csv', sep=";")
                df['image_path'] = image_frame_path + clip_folder + '/frames/'
                annotation_list.append(df)
        else:
            df = pd.read_csv(annotation_path +  'frameAnnotationsBOX.csv', sep=";")
            df['image_path'] = image_frame_path + 'frames/'
            annotation_list.append(df)
        
    df = pd.concat(annotation_list)
    df = df.drop(['Origin file', 'Origin frame number', 'Origin track', 'Origin track frame number'], axis=1)
    df.columns = ['filename', 'target', 'x1', 'y1', 'x2', 'y2', 'image_path']
    df = df[df['target'].isin(target_classes)]
    df['filename'] = df['filename'].apply(lambda filename: re.findall("\/([\d\w-]*.jpg)", filename)[0])
    df = df.drop_duplicates().reset_index(drop=True)
    return df

In [8]:
train_annotation_df = get_annotarion_dataframe(train_folder_list)

target_classes = train_annotation_df['target'].unique()
target_classes.sort()

train_annotation_df

,filename,target,x1,y1,x2,y2,image_path
0,daySequence1--00000.jpg,stop,706,478,718,500,../input/lisa-traffic-light-dataset/daySequenc...
1,daySequence1--00001.jpg,stop,705,475,720,497,../input/lisa-traffic-light-dataset/daySequenc...
2,daySequence1--00002.jpg,stop,707,476,719,494,../input/lisa-traffic-light-dataset/daySequenc...
3,daySequence1--00005.jpg,stop,708,474,720,492,../input/lisa-traffic-light-dataset/daySequenc...
4,daySequence1--00006.jpg,stop,707,470,722,492,../input/lisa-traffic-light-dataset/daySequenc...
...,...,...,...,...,...,...,...
14320,daySequence2--06881.jpg,go,388,0,448,50,../input/lisa-traffic-light-dataset/daySequenc...
14321,daySequence2--06881.jpg,go,1099,218,1144,283,../input/lisa-traffic-light-dataset/daySequenc...
14322,daySequence2--06882.jpg,go,1138,198,1183,273,../input/lisa-traffic-light-dataset/daySequenc...
14323,daySequence2--06883.jpg,go,1184,171,1232,256,../input/lisa-traffic-light-dataset/daySequenc...


In [9]:
def crop_image(image, x1, y1, x2, y2):
    cropped_image = image[y1:y2, x1:x2]
    cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
    
    return cropped_image

In [10]:
def generate_bouding_boxes(path):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Convert the image to a NumPy array
    image_array = np.array(image)

    # Convert the image array to L*a*b* color space
    lab_image = color.rgb2lab(image_array)

    # Extract the L, a, and b channels
    L = lab_image[:, :, 0]
    a = lab_image[:, :, 1]
    b = lab_image[:, :, 2]


    # Calculate the RGYB channel
    RGYB = 0.7*L * (1.0*a + 0.7*b)
    YELLOW = 0.7*L * (0.7*a + b)


    # Create a mask for red and green blobs
    red_mask = np.logical_and(RGYB > 2000, L <= 80)
    green_mask = np.logical_and(RGYB < -2000, L <= 80)
    yellow_mask = np.logical_and(YELLOW > 1000, RGYB <= 2000)

    # Apply the masks to the original image
    red_blobs = np.zeros_like(image_array)
    red_blobs[red_mask] = image_array[red_mask]
    green_blobs = np.zeros_like(image_array)
    green_blobs[green_mask] = image_array[green_mask]
    yellow_blobs = np.zeros_like(image_array)
    yellow_blobs[yellow_mask] = image_array[yellow_mask]

    # Convert the red and green blobs to grayscale
    red_gray = color.rgb2gray(red_blobs)
    green_gray = color.rgb2gray(green_blobs)
    yellow_gray = color.rgb2gray(yellow_blobs)

    red_gray = np.where(red_gray > 0, 255, 0).astype(np.uint8)
    green_gray = np.where(green_gray > 0, 255, 0).astype(np.uint8)
    yellow_gray = np.where(yellow_gray > 0, 255, 0).astype(np.uint8)


    # Use built-in sobel filter in hough circle transform.
    red_edges = red_gray
    green_edges = green_gray
    yellow_edges = yellow_gray

    # Perform Hough circle detection on red edges
    red_radii = np.arange(5, 30)  # Define the expected radius range for red circles
    red_hough = hough_circle(red_edges, red_radii)
    red_accums, red_centers_x, red_centers_y, red_radii = hough_circle_peaks(red_hough, red_radii, threshold=0.90*np.max(red_hough), min_xdistance = 50, min_ydistance = 50)

    # Perform Hough circle detection on green edges
    green_radii = np.arange(5, 30)  # Define the expected radius range for red circles
    green_hough = hough_circle(green_edges, green_radii)
    green_accums, green_centers_x, green_centers_y, green_radii = hough_circle_peaks(green_hough, green_radii, threshold=0.90*np.max(green_hough), min_xdistance=50, min_ydistance = 50)

    # Perform Hough circle detection on yellow edges
    yellow_radii = np.arange(5, 30)  # Define the expected radius range for yellow circles
    yellow_hough = hough_circle(yellow_edges, yellow_radii)
    yellow_accums, yellow_centers_x, yellow_centers_y, yellow_radii = hough_circle_peaks(yellow_hough, yellow_radii, threshold=0.90*np.max(yellow_hough), min_xdistance=50, min_ydistance = 50)


    # Draw detected circles on the original image

    cropped_images = []
    bounding_boxes = []
    labels = []

    red_circles_image = np.copy(image_array)
    for center_y, center_x, radius in zip(red_centers_y, red_centers_x, red_radii):
        circy, circx = circle_perimeter(center_y, center_x, radius)
        valid_coords = np.logical_and(circy >= 0, circy < red_circles_image.shape[0]) & np.logical_and(circx >= 0, circx < red_circles_image.shape[1])
        red_circles_image[circy[valid_coords], circx[valid_coords]] = (0, 255, 0)  # Red color for circles
        max_x = red_circles_image.shape[1]
        max_y = red_circles_image.shape[0]
        x1, y1, x2, y2 = max(0,center_x - 1.9*radius), max(0, center_y - 1.9*radius), min(max_x, center_x + 1.9*radius), min(max_y, center_y + (7.0)*radius)
        x1, x2, y1, y2 = round(x1), round(x2), round(y1), round(y2) 
        cropped_images.append(crop_image(image_array, x1, y1, x2, y2))
        bounding_boxes.append([x1, y1, x2, y2])
        labels.append("red")


    green_circles_image = np.copy(image_array)
    for center_y, center_x, radius in zip(green_centers_y, green_centers_x, green_radii):
        circy, circx = circle_perimeter(center_y, center_x, radius)
        valid_coords = np.logical_and(circy >= 0, circy < green_circles_image.shape[0]) & np.logical_and(circx >= 0, circx < green_circles_image.shape[1])
        green_circles_image[circy[valid_coords], circx[valid_coords]] = (255, 0, 0)  # Green color for circles
        max_x = green_circles_image.shape[1]
        max_y = green_circles_image.shape[0]
        x1, y1, x2, y2 = max(0,center_x - 1.9*radius), max(0, center_y - 7.0*radius), min(max_x, center_x + 1.9*radius), min(max_y, center_y + (1.9)*radius)
        x1, x2, y1, y2 = round(x1), round(x2), round(y1), round(y2) 
        cropped_images.append(crop_image(image_array, x1, y1, x2, y2))
        bounding_boxes.append([x1, y1, x2, y2])
        labels.append("green")
        
    yellow_circles_image = np.copy(image_array)
    for center_y, center_x, radius in zip(yellow_centers_y, yellow_centers_x, yellow_radii):
        circy, circx = circle_perimeter(center_y, center_x, radius)
        valid_coords = np.logical_and(circy >= 0, circy < yellow_circles_image.shape[0]) & np.logical_and(circx >= 0, circx < yellow_circles_image.shape[1])
        yellow_circles_image[circy[valid_coords], circx[valid_coords]] = (0, 0, 255)  # yellow color for circles
        max_x = yellow_circles_image.shape[1]
        max_y = yellow_circles_image.shape[0]
        x1, y1, x2, y2 = max(0,center_x - 1.9*radius), max(0, center_y - 4.1*radius), min(max_x, center_x + 1.0*radius), min(max_y, center_y + (4.1)*radius)
        x1, x2, y1, y2 = round(x1), round(x2), round(y1), round(y2) 
        cropped_images.append(crop_image(image_array, x1, y1, x2, y2))
        bounding_boxes.append([x1, y1, x2, y2])
        labels.append("yellow")
    
    return bounding_boxes

In [11]:
def crop_image(image, x1, y1, x2, y2):
    # cropped_image = image.crop((x1, y1, x2, y2))
    cropped_image = image[y1:y2, x1:x2, :]
    return cropped_image

In [12]:
unique_filenames = train_annotation_df['filename'].unique()
id_1 = 0
id_2 = 0

gt_annotations_list = []
pred_annotations_list = []
clf = joblib.load("clf.joblib")
test_percentage = 0.004
idx = 0


# Iterate over each unique filename
for filename in (unique_filenames):
    # Filter rows with the current filename
    random_number = np.random.rand()
    if (random_number > test_percentage):
        continue

    filtered_rows = train_annotation_df[train_annotation_df['filename'] == filename]
    img_path = filtered_rows.iloc[0]['image_path']
    img_name = filtered_rows.iloc[0]['filename']
    img = Image.open(os.path.join(img_path, img_name))

    current_gt_anotations = []
    current_pred_anotations = []
    X_test = []


    predicted_bounding_boxes = generate_bouding_boxes(img_path + img_name)
    for x1, y1, x2, y2 in predicted_bounding_boxes:
        cropped_image = img.crop((x1, y1, x2, y2))
        cropped_image = cropped_image.resize((20,34))
        cropped_image = np.array(cropped_image).flatten()  # Flatten the image
        cropped_image = cropped_image.astype(np.float64)/255
        current_pred_anotations.append([img_path+img_name, x1, y1, x2, y2])
        print(cropped_image.shape)
        predd = clf.predict([cropped_image])
        print(predd)
        X_test.append(cropped_image)

    if (len(X_test) == 0):
        continue
    idx+= 1
    y_pred = clf.predict(X_test).tolist()
    y_pred_prob = np.max(clf.predict_proba(X_test), axis = 1).tolist()
    concatenated_list = [a + [b, c] for a, b, c in zip((current_pred_anotations), (y_pred), (y_pred_prob))]

    pred_annotations_list.append(concatenated_list)


    for index, row in filtered_rows.iterrows():
        x1 = row['x1']
        y1 = row['y1']
        x2 = row['x2']
        y2 = row['y2']
        target = row['target']
        target = color_map[target]
        current_gt_anotations.append([img_path+img_name, x1, y1, x2, y2, target])
        # gt_annotations_list.append([x1, y1, x2, y2, target])
    gt_annotations_list.append(current_gt_anotations)


(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['yellow']
(2040,)
['red']
(2040,)
['not_traffic_light']
(2040,)
['red']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['red']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['yellow']
(2040,)
['yellow']
(2040,)
['red']
(2040,)
['not_traffic_light']
(2040,)
['red']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['red']
(2040,)
['not_traffic_light']
(2040,)
['red']
(2040,)
['red']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['green']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['not_traffic_light']
(2040,)
['green']
(2040,)
['green']
(2040,)
['n

In [13]:

# def evaluate(gt_annotations_list, pred_annotations_list):
    # Evaluate traffic light detection using ground truth and predicted annotations


gt_boxes_all_images = []
pred_boxes_all_images = []
confidence_scores_all_images = []
gt_classes_all_images = []
pred_classes_all_images = []

for i in range(len(gt_annotations_list)):
    gt_annotations = gt_annotations_list[i]
    pred_annotations = pred_annotations_list[i]

    gt_boxes = np.array([box[1:5] for box in gt_annotations])
    pred_boxes = np.array([box[1:5] for box in pred_annotations])
    confidence_scores = np.array([box[6] for box in pred_annotations])
    pred_classes = np.array([box[5] for box in pred_annotations])
    gt_classes = np.array([box[5] for box in gt_annotations])

    gt_matched = np.zeros(len(gt_boxes))

    if len(gt_boxes) > 0:
        for j in range(len(pred_boxes)):
            pred_box = pred_boxes[j]
            pred_class = pred_classes[j]
            max_iou = 0
            max_iou_index = -1

            for k in range(len(gt_boxes)):
                gt_box = gt_boxes[k]
                iou = calculate_iou(pred_box, gt_box)

                if iou > max_iou:
                    max_iou = iou
                    max_iou_index = k

            if max_iou >= 0.5 and not gt_matched[max_iou_index]:
                gt_matched[max_iou_index] = 1

                gt_boxes_all_images.append(gt_boxes[max_iou_index])
                pred_boxes_all_images.append(pred_box)
                confidence_scores_all_images.append(confidence_scores[j])
                gt_boxes_all_images.append(gt_boxes[max_iou_index])
                gt_classes_all_images.append(gt_classes[max_iou_index])
                pred_classes_all_images.append(pred_class)

gt_boxes_all_images = np.array(gt_boxes_all_images)
pred_boxes_all_images = np.array(pred_boxes_all_images)

# Calculate F1 score
for i in range(10):
    print("---")
    print(gt_annotations_list[i])
    print(pred_annotations_list[i])
f1 = f1_score(pred_classes_all_images, gt_classes_all_images, average='weighted')
print('F1 score:', f1)

---
[['../input/lisa-traffic-light-dataset/daySequence1/daySequence1/frames/daySequence1--00133.jpg', 756, 425, 774, 452, 'red']]
[['../input/lisa-traffic-light-dataset/daySequence1/daySequence1/frames/daySequence1--00133.jpg', 562, 516, 580, 560, 'not_traffic_light', 0.8403403695128541], ['../input/lisa-traffic-light-dataset/daySequence1/daySequence1/frames/daySequence1--00133.jpg', 494, 516, 514, 560, 'not_traffic_light', 0.7458701538261593], ['../input/lisa-traffic-light-dataset/daySequence1/daySequence1/frames/daySequence1--00133.jpg', 432, 424, 447, 464, 'not_traffic_light', 0.8005564172351962], ['../input/lisa-traffic-light-dataset/daySequence1/daySequence1/frames/daySequence1--00133.jpg', 494, 504, 509, 544, 'yellow', 0.5103810285273925]]
---
[['../input/lisa-traffic-light-dataset/daySequence1/daySequence1/frames/daySequence1--00759.jpg', 610, 381, 628, 408, 'red'], ['../input/lisa-traffic-light-dataset/daySequence1/daySequence1/frames/daySequence1--00759.jpg', 743, 434, 761, 46

In [1]:
evaluate_traffic_light_detection(gt_annotations_list, pred_annotations_list)

mAP@0.5:  0.3333333333333333
